In [1]:
# inputpath anpassen
datadir = "/home/chris/data/dlina/prose/fontane_output_norm/"

In [29]:
import os
from collections import Counter
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [30]:
rawfiles = os.listdir(datadir)

1) Einlesen von Dateien im CoNLL tab-Format, wie es die Kallimachos-Engine ausgibt (https://gitlab2.informatik.uni-wuerzburg.de/kallimachos/KallimachosEngines)

In [42]:
for f in rawfiles:
    name = f.split("_")[1]
    try:
        df = pd.read_csv(os.path.join(datadir, f), sep="\t", index_col=False)
        df.set_index(df.columns[:3].tolist(), inplace=True)
    except Exception as e:
        print(name)
        print(e)

effi-briest
Error tokenizing data. C error: EOF inside string starting at line 106696
der-stechlin
Error tokenizing data. C error: Expected 16 fields in line 28594, saw 21



In [44]:
df

TokenId   Begin     End               Token  \
SectionId ParagraphId SentenceId                                                
null      0           0                 0       0       3                 ﻿An   
                      0                 1       4       9               einem   
                      0                 2      10      13                 der   
                      0                 3      14      21             letzten   
                      0                 4      22      29             Maitage   
                      0                 5      29      30                   ,   
                      0                 6      31      34                 das   
                      0                 7      35      41              Wetter   
                      0                 8      42      45                 war   
                      0                 9      46      51               schon   
                      0                10      52      62          sommerlich   
                      0                11      62      63                   ,   
                      0                12      64      67                 bog   
                      0                13      68      71                 ein   
                      0                14      72      90  zurückgeschlagener   
                      0                15      91      99            Landauer   
                      0                16     100     103                 vom   
                      0                17     104     116        Spittelmarkt   
                      0                18     117     120                 her   
                      0                19     121     123                  in   
                      0                20     124     127                 die   
                      0                21     128     132                Kur-   
                      0                22     133     136                 und   
                      0                23     137     141                dann   
                      0                24     142     144                  in   
                      0                25     145     148                 die   
                      0                26     149     160         Adlerstraße   
                      0                27     161     164                 ein   
                      0                28     165     168                 und   
                      0                29     169     174               hielt   
...                                   ...     ...     ...                 ...   
          709         3139          72350  380886  380888                  es   
                      3139          72351  380889  380899          bestreitet   
                      3139          72352  380899  380900                   ,   
                      3139          72353  380901  380904                 ist   
                      3139          72354  380905  380908                 ein   
                      3139          72355  380909  380914               pecus   
                      3139          72356  380914  380915                   .   
                      3140          72357  380916  380921               Nicht   
                      3140          72358  380922  380926                wahr   
                      3140          72359  380926  380927                   ,   
                      3140          72360  380928  380931                 Kuh   
                      3140          72361  380931  380934                 ...   
                      3140          72362  380934  380935                   ?   
                      3140          72363  380936  380942              Kommen   
                      3140          72364  380943  380946                 Sie   
                      3140          72365  380946  380947                   ,   
                      3140          72366  380948  380953        

2) Ausgabe aller Named Entitys (aufsummiert nach Häufigkeit) im txt-Format. Diese Datei dient der manuelle Korrektur der Netzwerkdaten. 

Hier wird die entsprechende Funktion definiert die später auf jeden Roman angewendet wird. Übergeben wird der DataFrame sowie der Name des Romans, um das Ergebnis zu speichern.

In [7]:
def get_entity_counts(df, name):
    # first convert to Token column to a list and apply a Counter object to it
    NE_counts = Counter(df["Token"].tolist())
    # next we count the entities, sort by most common appearance
    # and transform it into a new dataframe with name and count columns
    NE_counts = pd.DataFrame(NE_counts.most_common(), columns=["NE_name", "count"])
    # last, we save the counts to a csv
    NE_counts.to_csv(os.path.join("results", name+"_entitycounts.csv"))

In [34]:
df

Begin                 End  \
SectionId ParagraphId SentenceId TokenId                               
0         0           0          0             3                 ﻿An   
                      1          4             9               einem   
                      2          10           13                 der   
                      3          14           21             letzten   
                      4          22           29             Maitage   
                      5          29           30                   ,   
                      6          31           34                 das   
                      7          35           41              Wetter   
                      8          42           45                 war   
                      9          46           51               schon   
                      10         52           62          sommerlich   
                      11         62           63                   ,   
                      12         64           67                 bog   
                      13         68           71                 ein   
                      14         72           90  zurückgeschlagener   
                      15         91           99            Landauer   
                      16         100         103                 vom   
                      17         104         116        Spittelmarkt   
                      18         117         120                 her   
                      19         121         123                  in   
                      20         124         127                 die   
                      21         128         132                Kur-   
                      22         133         136                 und   
                      23         137         141                dann   
                      24         142         144                  in   
                      25         145         148                 die   
                      26         149         160         Adlerstraße   
                      27         161         164                 ein   
                      28         165         168                 und   
                      29         169         174               hielt   
...                                          ...                 ...   
709       3139        72350      380886   380888                  es   
                      72351      380889   380899          bestreitet   
                      72352      380899   380900                   ,   
                      72353      380901   380904                 ist   
                      72354      380905   380908                 ein   
                      72355      380909   380914               pecus   
                      72356      380914   380915                   .   
          3140        72357      380916   380921               Nicht   
                      72358      380922   380926                wahr   
                      72359      380926   380927                   ,   
                      72360      380928   380931                 Kuh   
                      72361      380931   380934                 ...   
                      72362      380934   380935                   ?   
                      72363      380936   380942              Kommen   
                      72364      380943   380946                 Sie   
                      72365      380946   380947                   ,   
                      72366      380948   380953               meine   
                      72367      380954   380960              Herren   
                      72368      380960   380961                   ,   
                      72369      380962   380966                komm   
                      72370      380966   380967                   ,   
                      72371      380968   380973               Krola   
                      72372      380973   380976                 ...   
          3

3) Ausgabe von Relationslisten (d.i. Netzwerkdaten resp. Netzwerke) der Form „Akteur 1 – Akteur 2 – Häufigkeit“ als tsv-Datei, wobei für diese Relationslisten verschiedene Parametrisierungen vorgenommen werden können: 
* A) Akteure (also NEs) in einem Satz (abfragbar über sentenceID)
* B) Akteure (also NEs) in einem Absatz (abfragbar über paragraphID)
* C) Akteure (also NEs) in einem Absatz advanced, d.h. es werden nur Absätze mit mindestens einer bestimmten Anzahl von Wörtern (z.B. 500 oder 1000) berücksichtigt, wenn ein Absatz länger ist, wird der folgende dazugezählt. Die Anzahl der Wörter sollte einstellbar sein.
* D) Akteure (also NEs) in einem Kapitel (abfragbar über sectionID)

In [19]:
def create_network(df, name, aggregation):
    """Aggregations should be one of ["SectionId, ParagraphId, SentenceId]"""
    # first we group the tokens by the desired level of aggregation and count them
    ne_occurrences = df.groupby(aggregation)["Token"].count()
    # next we filter out any tokens that only appear alone
    # we then extract the indexes where entities co-occur
    ne_cooc_inds = ne_occurrences[ne_occurrences > 1].index.tolist()
    # next follows a helper function that gives us the correct index to extract the entities
    islice = get_index_slice(ne_cooc_inds, aggregation)
    # next we group the co-occuring entities by the desired aggregation level
    # and transform the grouped entities to lists
    segments = (df.loc[islice, :]["Token"]
                  .groupby(aggregation)
                  .apply(lambda x: list(x)))
    # in the next block we create the network
    B = nx.Graph()
    # we iterate over the segments (sections/paragraphs/sentences)
    # items are co-occurring entities
    for s, items in segments.iteritems():
        source = str(s)
        targets = items
        if source not in B.nodes():
            B.add_node(source, bipartite=0)
        
        for target in targets:
            if target not in B.nodes():
                B.add_node(target, bipartite=1)
            B.add_edge(source, target)
    
    segment_nodes = set(n
                        for n, d in B.nodes(data=True)
                        if d['bipartite'] == 0)
    entity_nodes = set(B) - segment_nodes
    nx.is_bipartite(B)
    # last, we project the bipartite graph of segments & entities into a an entity-cooccurrence graph
    G = nx.bipartite.weighted_projected_graph(B, entity_nodes)
    return G
    
def get_index_slice(ne_coocs_inds, aggregation):
    if aggregation == "SectionId":
        islice = pd.IndexSlice[ne_cooc_inds, :, :]
    if aggregation == "ParagraphId":
        islice = pd.IndexSlice[:, ne_cooc_inds, :]        
    if aggregation == "SentenceId":
        islice = pd.IndexSlice[:, :, ne_cooc_inds]      
    return islice

4) Ausgabe von Netzwerkmetriken, wie sie z.B. vom Tool dramavis ausgegeben werden, für die unter 3) genannten Netzwerke. 

In [21]:
def get_character_metrics(G):
    personae = G.nodes()
    centralities = pd.DataFrame(index = [p for p in personae])
    centralities.index.name = "name"
    for metric in ['betweenness', 'degree',
                   'closeness', 'closeness_corrected',
                   'strength',
                   'eigenvector_centrality']:
        centralities[metric] = 0
    for char, metric in nx.betweenness_centrality(G).items():
        centralities.loc[char, 'betweenness'] = metric
    for char, metric in nx.degree(G).items():
        centralities.loc[char, 'degree'] = metric
    for char, metric in nx.degree(G, weight="weight").items():
        centralities.loc[char, 'strength'] = metric
    for char, metric in nx.closeness_centrality(G).items():
        centralities.loc[char, 'closeness'] = metric
    for g in nx.connected_component_subgraphs(G):
        for char, metric in nx.closeness_centrality(g).items():
            centralities.loc[char, 'closeness_corrected'] = metric
    try:
        for char, metric in nx.eigenvector_centrality(G,
                                            max_iter=500).items():
            centralities.loc[char, 'eigenvector_centrality'] = metric
    except Exception as e:
        pass
    centralities['avg_distance'] = 1/centralities['closeness']
    centralities['avg_distance_corrected'] = 1/centralities['closeness_corrected']
    return centralities

In [22]:
def get_graph_metrics(G):
    values = {}
    values["charcount"] = len(G.nodes())
    values["edgecount"] = len(G.edges())
    try:
        values["maxdegree"] = max(G.degree().values())
    except:
        print("ValueError: max() arg is an empty sequence")
        values["maxdegree"] = "NaN"

    try:
        values["avgdegree"] = sum(G.degree().values())/len(G.nodes())
    except:
        values["avgdegree"] = "NaN"

    try:
        values["density"] = nx.density(G)
    except:
        values["density"] = "NaN"

    try:
        values["avgpathlength"] = nx.average_shortest_path_length(G)
    except nx.NetworkXError:
        try:
            self.randomization = 50
            values["avgpathlength"] = nx.average_shortest_path_length(
                        max(nx.connected_component_subgraphs(G), key=len))
        except:
            values["avgpathlength"] = "NaN"
    except:
        values["avgpathlength"] = "NaN"

    try:
        values["clustering_coefficient"] = nx.average_clustering(G)
    except:
        values["clustering_coefficient"] = "NaN"
    values["connected_components"] = nx.number_connected_components(G)
    components = nx.connected_component_subgraphs(G)
    values["component_sizes"] = [len(c.nodes()) for c in components]
    graph_metrics = pd.DataFrame.from_dict(values)
    return graph_metrics

In [23]:
def plotGraph(G, figsize=(8, 8), filename=None):
    """
    Plots an individual graph, node size by degree centrality,
    edge size by edge weight.
    """
    labels = {n:n for n in G.nodes()}

    try:
        # for networks with only one node
        d = nx.degree_centrality(G)
        nodesize = [v * 250 for v in d.values()]
    except:
        nodesize = [1 * 250 for n in G.nodes()]

    layout=nx.spring_layout
    pos=layout(G)

    plt.figure(figsize=figsize)
    plt.subplots_adjust(left=0,right=1,bottom=0,top=0.95,wspace=0.01,hspace=0.01)

    # nodes
    nx.draw_networkx_nodes(G,pos,
                            nodelist=G.nodes(),
                            node_color="steelblue",
                            node_size=nodesize,
                            alpha=0.8)
    try:
        weights = [G[u][v]['weight'] for u,v in G.edges()]
    except:
        weights = [1 for u,v in G.edges()]
    nx.draw_networkx_edges(G,pos,
                           with_labels=False,
                           edge_color="grey",
                           width=weights
                        )

    if G.order() < 1000:
        nx.draw_networkx_labels(G,pos, labels)
    plt.savefig(filename)
    plt.close("all")

In [50]:
######
# set aggregation level here, SectionId, ParagraphId, SentenceId
aggregation = "SentenceId"
######
for f in rawfiles:
    name = f.split("_")[1]
    try:
        df = pd.read_csv(os.path.join(datadir, f), sep="\t", index_col=False)
        df.set_index(df.columns[:3].tolist(), inplace=True)
        # we filter the token dataframe to include only tokens with CPOS==NE
        df = df[df["POS"] == "NE"]
        # Anwendung von 2)
        get_entity_counts(df, name)
        # Anwendung von 3)
        G = create_network(df, name, aggregation)
        # Berechnung und Speicherung von 4)
        char_metrics = get_character_metrics(G)
        graph_metrics = get_graph_metrics(G)
        char_metrics.to_csv(os.path.join("results",
                                         "_".join([name,
                                                   aggregation,
                                                   "char_metrics.csv"])))
        graph_metrics.to_csv(os.path.join("results",
                                         "_".join([name,
                                                   aggregation,
                                                   "graph_metrics.csv"])))
        plotGraph(G, filename=os.path.join("results",
                                           "_".join([name,
                                                     aggregation+".svg"])))
    except Exception as e:
        print(": ".join([name, str(e)]))

effi-briest: Error tokenizing data. C error: EOF inside string starting at line 106696
der-stechlin: Error tokenizing data. C error: Expected 16 fields in line 28594, saw 21

